# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f7cc4e20b50>
├── 'a' --> tensor([[-0.1354, -1.1270, -0.7803],
│                   [ 0.1163,  0.5671,  1.6967]])
└── 'x' --> <FastTreeValue 0x7f7cc4e20c40>
    └── 'c' --> tensor([[-2.6757,  0.4287, -0.9984,  0.9214],
                        [ 0.1942,  1.1842,  1.0893,  0.1120],
                        [ 1.2260, -0.4942, -0.6620,  0.5195]])

In [4]:
t.a

tensor([[-0.1354, -1.1270, -0.7803],
        [ 0.1163,  0.5671,  1.6967]])

In [5]:
%timeit t.a

61.8 ns ± 0.949 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f7cc4e20b50>
├── 'a' --> tensor([[-0.3031, -1.4068, -1.3145],
│                   [-1.5583, -1.1716, -0.9380]])
└── 'x' --> <FastTreeValue 0x7f7cc4e20c40>
    └── 'c' --> tensor([[-2.6757,  0.4287, -0.9984,  0.9214],
                        [ 0.1942,  1.1842,  1.0893,  0.1120],
                        [ 1.2260, -0.4942, -0.6620,  0.5195]])

In [7]:
%timeit t.a = new_value

58.3 ns ± 0.0829 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.1354, -1.1270, -0.7803],
               [ 0.1163,  0.5671,  1.6967]]),
    x: Batch(
           c: tensor([[-2.6757,  0.4287, -0.9984,  0.9214],
                      [ 0.1942,  1.1842,  1.0893,  0.1120],
                      [ 1.2260, -0.4942, -0.6620,  0.5195]]),
       ),
)

In [10]:
b.a

tensor([[-0.1354, -1.1270, -0.7803],
        [ 0.1163,  0.5671,  1.6967]])

In [11]:
%timeit b.a

53.3 ns ± 0.041 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.0122, -0.3830,  2.1192],
               [-0.5294,  0.2058,  1.6641]]),
    x: Batch(
           c: tensor([[-2.6757,  0.4287, -0.9984,  0.9214],
                      [ 0.1942,  1.1842,  1.0893,  0.1120],
                      [ 1.2260, -0.4942, -0.6620,  0.5195]]),
       ),
)

In [13]:
%timeit b.a = new_value

479 ns ± 0.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

846 ns ± 0.0881 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.6 µs ± 16.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

140 µs ± 631 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 170 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f7c0f0bd100>
├── 'a' --> tensor([[[-0.1354, -1.1270, -0.7803],
│                    [ 0.1163,  0.5671,  1.6967]],
│           
│                   [[-0.1354, -1.1270, -0.7803],
│                    [ 0.1163,  0.5671,  1.6967]],
│           
│                   [[-0.1354, -1.1270, -0.7803],
│                    [ 0.1163,  0.5671,  1.6967]],
│           
│                   [[-0.1354, -1.1270, -0.7803],
│                    [ 0.1163,  0.5671,  1.6967]],
│           
│                   [[-0.1354, -1.1270, -0.7803],
│                    [ 0.1163,  0.5671,  1.6967]],
│           
│                   [[-0.1354, -1.1270, -0.7803],
│                    [ 0.1163,  0.5671,  1.6967]],
│           
│                   [[-0.1354, -1.1270, -0.7803],
│                    [ 0.1163,  0.5671,  1.6967]],
│           
│                   [[-0.1354, -1.1270, -0.7803],
│                    [ 0.1163,  0.5671,  1.6967]]])
└── 'x' --> <FastTreeValue 0x7f7c0f0bd160>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.8 µs ± 83.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f7cc4e204c0>
├── 'a' --> tensor([[-0.1354, -1.1270, -0.7803],
│                   [ 0.1163,  0.5671,  1.6967],
│                   [-0.1354, -1.1270, -0.7803],
│                   [ 0.1163,  0.5671,  1.6967],
│                   [-0.1354, -1.1270, -0.7803],
│                   [ 0.1163,  0.5671,  1.6967],
│                   [-0.1354, -1.1270, -0.7803],
│                   [ 0.1163,  0.5671,  1.6967],
│                   [-0.1354, -1.1270, -0.7803],
│                   [ 0.1163,  0.5671,  1.6967],
│                   [-0.1354, -1.1270, -0.7803],
│                   [ 0.1163,  0.5671,  1.6967],
│                   [-0.1354, -1.1270, -0.7803],
│                   [ 0.1163,  0.5671,  1.6967],
│                   [-0.1354, -1.1270, -0.7803],
│                   [ 0.1163,  0.5671,  1.6967]])
└── 'x' --> <FastTreeValue 0x7f7cc4e20940>
    └── 'c' --> tensor([[-2.6757,  0.4287, -0.9984,  0.9214],
                        [ 0.1942,  1.1842,  1.0893,  0.1120],
                 

In [23]:
%timeit t_cat(trees)

29.5 µs ± 49 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59 µs ± 103 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-2.6757,  0.4287, -0.9984,  0.9214],
                       [ 0.1942,  1.1842,  1.0893,  0.1120],
                       [ 1.2260, -0.4942, -0.6620,  0.5195]],
              
                      [[-2.6757,  0.4287, -0.9984,  0.9214],
                       [ 0.1942,  1.1842,  1.0893,  0.1120],
                       [ 1.2260, -0.4942, -0.6620,  0.5195]],
              
                      [[-2.6757,  0.4287, -0.9984,  0.9214],
                       [ 0.1942,  1.1842,  1.0893,  0.1120],
                       [ 1.2260, -0.4942, -0.6620,  0.5195]],
              
                      [[-2.6757,  0.4287, -0.9984,  0.9214],
                       [ 0.1942,  1.1842,  1.0893,  0.1120],
                       [ 1.2260, -0.4942, -0.6620,  0.5195]],
              
                      [[-2.6757,  0.4287, -0.9984,  0.9214],
                       [ 0.1942,  1.1842,  1.0893,  0.1120],
                       [ 1.2260, -0.4942, -0.6620,  0.5195]],

In [26]:
%timeit Batch.stack(batches)

77.7 µs ± 72 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-2.6757,  0.4287, -0.9984,  0.9214],
                      [ 0.1942,  1.1842,  1.0893,  0.1120],
                      [ 1.2260, -0.4942, -0.6620,  0.5195],
                      [-2.6757,  0.4287, -0.9984,  0.9214],
                      [ 0.1942,  1.1842,  1.0893,  0.1120],
                      [ 1.2260, -0.4942, -0.6620,  0.5195],
                      [-2.6757,  0.4287, -0.9984,  0.9214],
                      [ 0.1942,  1.1842,  1.0893,  0.1120],
                      [ 1.2260, -0.4942, -0.6620,  0.5195],
                      [-2.6757,  0.4287, -0.9984,  0.9214],
                      [ 0.1942,  1.1842,  1.0893,  0.1120],
                      [ 1.2260, -0.4942, -0.6620,  0.5195],
                      [-2.6757,  0.4287, -0.9984,  0.9214],
                      [ 0.1942,  1.1842,  1.0893,  0.1120],
                      [ 1.2260, -0.4942, -0.6620,  0.5195],
                      [-2.6757,  0.4287, -0.9984,  0.9214],
                   

In [28]:
%timeit Batch.cat(batches)

142 µs ± 342 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

313 µs ± 3.44 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
